In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.python.client import device_lib
from keras.models import load_model

import matplotlib.pyplot as plt
import numpy as np  
import math
import pandas as pd

import backtrader as bt
from arch import arch_model
%matplotlib inline

In [3]:
def custom_mae_loss(y_true, y_pred):
    y_true_next = tf.cast(y_true[:, 1], tf.float64)  # Extract the true next values, scaled
    y_pred_next = tf.cast(y_pred[:, 0], tf.float64)  # Extract the predicted next values, scaled
    abs_error = tf.abs(y_true_next - y_pred_next)  # Calculate the absolute error
    return tf.reduce_mean(abs_error)  # Return the mean of these errors

def dir_acc(y_true, y_pred):
    mean, std = tf.cast(y_true[:, 2], tf.float64), tf.cast(y_true[:, 3], tf.float64)  # Retrieve scaling factors
    y_true_prev = (tf.cast(y_true[:, 0], tf.float64) * std) + mean  # Un-scale previous true price
    y_true_next = (tf.cast(y_true[:, 1], tf.float64) * std) + mean  # Un-scale next true price
    y_pred_next = (tf.cast(y_pred[:, 0], tf.float64) * std) + mean  # Un-scale predicted next price

    true_change = y_true_next - y_true_prev  # Calculate true change
    pred_change = y_pred_next - y_true_prev  # Calculate predicted change

    correct_direction = tf.equal(tf.sign(true_change), tf.sign(pred_change))  # Check if the signs match
    return tf.reduce_mean(tf.cast(correct_direction, tf.float64))  # Return the mean of correct directions


def get_lr_callback(batch_size=16, mode='cos', epochs=500, plot=False):
    lr_start, lr_max, lr_min = 0.0001, 0.005, 0.00001  # Adjust learning rate boundaries
    lr_ramp_ep = int(0.30 * epochs)  # 30% of epochs for warm-up
    lr_sus_ep = max(0, int(0.10 * epochs) - lr_ramp_ep)  # Optional sustain phase, adjust as needed

    def lrfn(epoch):
        if epoch < lr_ramp_ep:  # Warm-up phase
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep:  # Sustain phase at max learning rate
            lr = lr_max
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        else:
            lr = lr_min  # Default to minimum learning rate if mode is not recognized

        return lr

    if plot:  # Plot learning rate curve if plot is True
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.title('Learning Rate Scheduler')
        plt.show()

    return tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

In [4]:
from keras.models import load_model

model_path = "/Users/damensavvasavvi/Desktop/AlgoTrading/Transformers/DirectionalPredictions/googleColab/transformer_val_model.keras"
transformer_model = load_model(model_path, custom_objects={"custom_mae_loss": custom_mae_loss, "dir_acc": dir_acc, "get_lr_callback": get_lr_callback})

2025-08-22 17:13:47.817853: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-08-22 17:13:47.817884: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-08-22 17:13:47.817890: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2025-08-22 17:13:47.817920: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-08-22 17:13:47.817929: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/opt/miniconda3/envs/tf_py3.12.6/lib/python3.12/site-packages/keras/src/trainers/trainer.py:212: UserWarning: Model doesn't support `jit_compile=True`. Proceeding with `jit_compile=False

In [5]:
file = '/users/damensavvasavvi/Desktop/AlgoTrading/marketDataNasdaqFutures/NQ_continuous_backadjusted_1m_cleaned.csv'
df = pd.read_csv(file,usecols=['open','high','low','close','volume','timestamp'], parse_dates=['timestamp'])
# df = df[(df['timestamp'].dt.month == 1) & (df['timestamp'].dt.year == 2025)]  # Filter for January 2025
df = df[ (df['timestamp'].dt.year == 2025)]
df.rename(columns={'timestamp': 'datetime'}, inplace=True)
df = df[::5]
df

from scipy import stats
import numpy as np

z_scores = np.abs(stats.zscore(df['close']))
outliers = df[z_scores > 3]  # Values >3 standard deviations
print("Outliers detected:", len(outliers))
pct_change = df['close'].pct_change().abs()
outliers = df[pct_change > 0.5]  # >50% sudden jumps
print("Suspicious jumps:", outliers[['close']].head())
df['close'] = df['close'].where(
    (z_scores <= 3) & (pct_change <= 0.5), 
    np.nan
).interpolate()  # Linear interpolation

Outliers detected: 266
Suspicious jumps: Empty DataFrame
Columns: [close]
Index: []


In [6]:

features = pd.read_csv('/Users/damensavvasavvi/Desktop/AlgoTrading/marketDataNasdaqFutures/features.csv', index_col=0, parse_dates=True)
stats = pd.read_csv('/Users/damensavvasavvi/Desktop/AlgoTrading/marketDataNasdaqFutures/stats.csv', index_col=0)


/var/folders/mv/z1xz77z55w58nq2fljfj8dvh0000gn/T/ipykernel_59333/1217933948.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  features = pd.read_csv('/Users/damensavvasavvi/Desktop/AlgoTrading/marketDataNasdaqFutures/features.csv', index_col=0, parse_dates=True)


In [7]:
from backtesting import Strategy
import numpy as np
from keras.models import load_model

prediction_timeseries = []

class TransformerStrategy(Strategy):
    # Strategy parameters
    prediction_sequence_length = 24
    stop_loss_amount           = 50.0
    take_profit_amount         = 150.0
    fixed_contract_size        = 5
    commission_per_contract    = 2.0
    tick_value                 = 5.0
    stop_loss_ticks            = 100   # 50 ticks stop loss
    take_profit_ticks          = 200
    tick_size                  = 0.25
    trade_start_hour           = 7
    trade_end_hour             = 18
    trade_start_minute         = 0
    trade_end_minute           = 0

    def log(self, txt):
        """Simple logger using current bar’s timestamp."""
        dt = self.data.index[-1]
        print(f"{dt}, {txt}")

    def init(self):
        # External globals: features, stats, custom loss/metrics
        self.features         = features
        self.TransformerModel = load_model(
            "/Users/damensavvasavvi/Desktop/AlgoTrading/Transformers/"
            "DirectionalPredictions/googleColab/transformer_val_model.keras",
            custom_objects={
                "custom_mae_loss": custom_mae_loss,
                "dir_acc":         dir_acc,
                "get_lr_callback": get_lr_callback
            }
        )

        self.prediction          = None
        self.last_prediction_bar = self.trade_start_hour * 12
        self.log(f"Initialized; last_prediction_bar set to {self.last_prediction_bar}")

    def _calculate_bracket_prices(self, current_price, is_long):
        sl_move = self.stop_loss_ticks  * self.tick_size
        tp_move = self.take_profit_ticks * self.tick_size

        if is_long:
            stop_price        = current_price - sl_move
            take_profit_price = current_price + tp_move
        else:
            stop_price        = current_price + sl_move
            take_profit_price = current_price - tp_move

        # Round to nearest tick
        stop_price        = round(stop_price / self.tick_size) * self.tick_size
        take_profit_price = round(take_profit_price / self.tick_size) * self.tick_size

        return stop_price, take_profit_price

    def next(self):
        # Need enough bars to form a prediction sequence
        if len(self.data.Close) < self.prediction_sequence_length:
            return

        # Trigger prediction halfway through the sequence interval
        if (len(self.data.Close) - self.last_prediction_bar
            ) == (self.prediction_sequence_length // 2):

            self.log(f"Making prediction at bar {len(self.data.Close)}")

            # Prepare model input
            seq_slice = slice(
                len(self.data.Close) - self.prediction_sequence_length,
                len(self.data.Close)
            )
            X = np.expand_dims(self.features[seq_slice], axis=0)

            # Predict & denormalize using external stats
            nxt = self.TransformerModel.predict(X, verbose=0)[0][0]
            nxt = nxt * stats['dataClose'][1] + stats['dataClose'][0]
            prediction_timeseries.append(nxt)
            self.log(f"PREDICTION: {nxt:.2f}")

            price = float(self.data.Close[-1])

            # Initialize first prediction
            if self.prediction is None:
                self.prediction          = nxt
                self.last_prediction_bar = len(self.data.Close)
                self.log(f"Initial prediction set to {nxt:.2f}")
                return

            # Compare to previous prediction
            if nxt > self.prediction:
                self.log(f"BUY SIGNAL: predicted {nxt:.2f} > {self.prediction:.2f}")

                # Close any short
                if self.position and self.position.size < 0:
                    self.position.close()

                # Place long bracket if flat
                if not self.position:
                    sl, tp = self._calculate_bracket_prices(price, is_long=True)
                    self.log(f"Placing BUY bracket: entry {price:.2f}, SL {sl:.2f}, TP {tp:.2f}")
                    self.buy(size=self.fixed_contract_size, sl=sl, tp=tp)

            elif nxt < self.prediction:
                self.log(f"SELL SIGNAL: predicted {nxt:.2f} < {self.prediction:.2f}")

                # Close any long
                if self.position and self.position.size > 0:
                    self.position.close()

                # Place short bracket if flat
                if not self.position:
                    sl, tp = self._calculate_bracket_prices(price, is_long=False)
                    self.log(f"Placing SELL bracket: entry {price:.2f}, SL {sl:.2f}, TP {tp:.2f}")
                    self.sell(size=self.fixed_contract_size, sl=sl, tp=tp)

            # Update state
            self.prediction          = nxt
            self.last_prediction_bar = len(self.data.Close)
            self.log(f"Updated last_prediction_bar to {self.last_prediction_bar}")


/opt/miniconda3/envs/tf_py3.12.6/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [8]:
df = df[df['datetime'].dt.month >3]

In [9]:
%matplotlib widget
import itertools

import matplotlib.pyplot as plt
from backtesting import Backtest

# Assume `df`, `features`, `stats` and `TransformerStrategy` are already defined
# df must have columns: datetime, open, high, low, close, volume

# 1. Prepare DataFrame for backtesting.py

df.rename(columns={
        'open':   'Open',
        'high':   'High',
        'low':    'Low',
        'close':  'Close',
        'volume': 'Volume'
    },inplace= True)
df.interpolate(method='linear', inplace=True)
df.dropna(inplace=True)

print("Data prepared for backtesting:", df)
# 2. Compute fixed commission per trade (fixed_contract_size * commission_per_contract)
commission_per_trade = (
   0.002
)

# 3. Instantiate and run the backtest
bt = Backtest(
    df,
    TransformerStrategy,
    cash=100_000,
    commission=commission_per_trade,
    exclusive_orders=True
)

stats = bt.run()
print(stats)
# 4. Print portfolio values
print(f"Starting Portfolio Value: {stats['Start']:.2f}")
print(f"Final Portfolio Value:    {stats['End']:.2f}\n")

# # 5. Extract trade-by-trade results
# trades_df = stats._trades.copy()

# total_trades = len(trades_df)
# wins         = (trades_df['Return [%]'] > 0).sum()
# losses       = (trades_df['Return [%]'] < 0).sum()
# win_rate     = wins / total_trades * 100 if total_trades else 0
# loss_rate    = losses / total_trades * 100 if total_trades else 0

# # Compute longest win/loss streak
# flags         = (trades_df['Return [%]'] > 0).astype(int)
# win_streaks   = [sum(1 for _ in grp) for val, grp in itertools.groupby(flags) if val == 1]
# loss_streaks  = [sum(1 for _ in grp) for val, grp in itertools.groupby(flags) if val == 0]
# longest_win   = max(win_streaks) if win_streaks else 0
# longest_loss  = max(loss_streaks) if loss_streaks else 0

# print('--- Trade Analyzer Results ---')
# print(f'Total Trades:     {total_trades}')
# print(f'Won Trades:       {wins}')
# print(f'Lost Trades:      {losses}')
# print(f'Win Rate:         {win_rate:.2f}%')
# print(f'Loss Rate:        {loss_rate:.2f}%')
# print(f'Longest Win Streak:  {longest_win}')
# print(f'Longest Loss Streak: {longest_loss}\n')

# 6. Plot results (interactive widget)
bt.plot(plot_volume=True)
plt.show()


/opt/miniconda3/envs/tf_py3.12.6/lib/python3.12/site-packages/backtesting/backtesting.py:1212: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  (data.index.is_numeric() and
/var/folders/mv/z1xz77z55w58nq2fljfj8dvh0000gn/T/ipykernel_59333/164728862.py:29: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(


Data prepared for backtesting:                          datetime          Open          High           Low  \
1638496 2025-04-01 00:03:00+00:00  19618.778028  19625.626212  19618.270756   
1638501 2025-04-01 00:08:00+00:00  19625.372576  19637.800761  19624.865303   
1638506 2025-04-01 00:13:00+00:00  19623.850757  19626.133485  19619.538938   
1638511 2025-04-01 00:18:00+00:00  19635.264396  19643.127126  19635.010760   
1638516 2025-04-01 00:23:00+00:00  19638.054397  19641.605307  19635.264396   
...                           ...           ...           ...           ...   
1741981 2025-07-17 23:39:00+00:00  23268.500000  23269.000000  23266.500000   
1741986 2025-07-17 23:44:00+00:00  23271.250000  23272.000000  23269.500000   
1741991 2025-07-17 23:49:00+00:00  23275.250000  23277.000000  23274.750000   
1741996 2025-07-17 23:54:00+00:00  23284.500000  23284.500000  23279.750000   
1742001 2025-07-17 23:59:00+00:00  23283.500000  23285.500000  23280.500000   

                Clos

/opt/miniconda3/envs/tf_py3.12.6/lib/python3.12/site-packages/keras/src/trainers/trainer.py:212: UserWarning: Model doesn't support `jit_compile=True`. Proceeding with `jit_compile=False`.
  warnings.warn(
/opt/miniconda3/envs/tf_py3.12.6/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 394 variables whereas the saved optimizer has 398 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1742001, Initialized; last_prediction_bar set to 84
1638971, Making prediction at bar 96


2025-08-22 17:13:54.598959: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1638971, PREDICTION: 8896.58
1638971, Initial prediction set to 8896.58
1639031, Making prediction at bar 108
1639031, PREDICTION: 8885.40
1639031, SELL SIGNAL: predicted 8885.40 < 8896.58
1639031, Placing SELL bracket: entry 19752.70, SL 19777.75, TP 19702.75
1639031, Updated last_prediction_bar to 108
1639091, Making prediction at bar 120
1639091, PREDICTION: 8924.52
1639091, BUY SIGNAL: predicted 8924.52 > 8885.40
1639091, Placing BUY bracket: entry 19775.27, SL 19750.25, TP 19825.25
1639091, Updated last_prediction_bar to 120
1639151, Making prediction at bar 132
1639151, PREDICTION: 9097.78
1639151, BUY SIGNAL: predicted 9097.78 > 8924.52
1639151, Placing BUY bracket: entry 19685.74, SL 19660.75, TP 19735.75
1639151, Updated last_prediction_bar to 132
1639211, Making prediction at bar 144
1639211, PREDICTION: 9041.89
1639211, SELL SIGNAL: predicted 9041.89 < 9097.78
1639211, Placing SELL bracket: entry 19639.58, SL 19664.50, TP 19589.50
1639211, Updated last_prediction_bar to 144
